In [1]:
!pip install transformers==4.28.0
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 16.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
!pip install datasets
!pip install nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset
import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

# Uncomment the line below if you need to download NLTK resources
# nltk.download('punkt')

# Load the pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

# Define a function to generate text from tabular data
def generate_text(row):
    timestamp = row['Timestamp']
    participant_id = row['Participant_ID']
    gender = row['Gender']
    nationality = row['Nationality']
    age = row['Age']
    food = row['Food']
    juice = row['Juice']
    dessert = row['Dessert']

    text = f"Timestamp: {timestamp}\nParticipant ID: {participant_id}\nGender: {gender}\nNationality: {nationality}\nAge: {age}\nFood: {food}\nJuice: {juice}\nDessert: {dessert}\n"
    return text

# Load and preprocess the dataset
df = pd.read_csv("/content/food.csv", nrows=20)  # Update with your dataset file path

# Split the dataset into training and evaluation sets
train_size = int(0.8 * len(df))
train_df = df[:train_size]
eval_df = df[train_size:]

text_data_train = []
text_data_eval = []

# Use NLTK for tokenization
for _, row in train_df.iterrows():
    text = generate_text(row)
    tokens = word_tokenize(text)
    text_data_train.append(" ".join(tokens))

for _, row in eval_df.iterrows():
    text = generate_text(row)
    tokens = word_tokenize(text)
    text_data_eval.append(" ".join(tokens))

# Tokenize and create datasets
tokenized_data_train = tokenizer(text_data_train, truncation=True, padding='max_length', max_length=128, return_tensors="pt")
tokenized_data_eval = tokenizer(text_data_eval, truncation=True, padding='max_length', max_length=128, return_tensors="pt")

train_dataset = Dataset.from_dict(tokenized_data_train)
eval_dataset = Dataset.from_dict(tokenized_data_eval)

# Configure and train the model using the Trainer class
training_args = TrainingArguments(
    output_dir="output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_steps=100,
    save_steps=100,
    warmup_steps=0,
    logging_dir="logs",
    evaluation_strategy="steps",
    save_total_limit=3,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

trainer.train()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss


TrainOutput(global_step=6, training_loss=3.307440439860026, metrics={'train_runtime': 136.3212, 'train_samples_per_second': 0.352, 'train_steps_per_second': 0.044, 'total_flos': 3135504384000.0, 'train_loss': 3.307440439860026, 'epoch': 3.0})

In [4]:
# Save the fine-tuned model
model.save_pretrained("fine_tuned_gpt2")
tokenizer.save_pretrained("fine_tuned_gpt2_tokenizer")


('fine_tuned_gpt2_tokenizer/tokenizer_config.json',
 'fine_tuned_gpt2_tokenizer/special_tokens_map.json',
 'fine_tuned_gpt2_tokenizer/vocab.json',
 'fine_tuned_gpt2_tokenizer/merges.txt',
 'fine_tuned_gpt2_tokenizer/added_tokens.json',
 'fine_tuned_gpt2_tokenizer/tokenizer.json')

In [5]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# Load the fine-tuned model
fine_tuned_model = GPT2LMHeadModel.from_pretrained("fine_tuned_gpt2")

# Load the fine-tuned tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("fine_tuned_gpt2_tokenizer")


def ask_question(question, model, tokenizer, max_length=128, num_return_sequences=1):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True)
    outputs = model.generate(
        inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=3,
        do_sample=True,
        temperature=1.0,
        top_k=50,
        top_p=0.9,
        early_stopping=True,
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = answer.replace(prompt, "").strip()

    # Truncate the answer after the first newline character
    answer = answer.split("\n")[0]

    return answer

# Ask questions using the fine-tuned model based on user input
while True:
    user_question = input("Ask a question (or type 'exit' to end): ")

    if user_question.lower() == 'exit':
        print("Goodbye!")
        break

    answer = ask_question(user_question, fine_tuned_model, tokenizer)
    print(f"Answer: {answer}\n")


Ask a question (or type 'exit' to end): hi


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: So you see my question is... what are the steps of this?

Ask a question (or type 'exit' to end): how many cusines are there in the dataset


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Answer: It is very low, mainly because the dataset is mainly in Latin America. The authors were able to find cusine count in Bolivia by searching for Spanish language languages and Latin American language populations using their own search techniques. Unfortunately they did not find Cuzine which were mainly from South America.

Ask a question (or type 'exit' to end): exit
Goodbye!
